In [ ]:
## -- This notebook is for understanding systematics data frame makers
#!/usr/bin/env python3 
import sys
import os
import pandas as pd
import uproot
import warnings

# Add the head direcoty to sys.path
workspace_root = os.getcwd()  
sys.path.insert(0, workspace_root + "/../../")

# import this repo's classes
from pyanalib.ntuple_glob import NTupleGlob
import pyanalib.pandas_helpers as ph
import makedf.branches as br
from makedf.makedf import *
from makedf.geniesyst import *
from makedf.getsyst import *


In [ ]:
test_sample_path = "root://fndca1.fnal.gov:1094/pnfs/fnal.gov/usr/sbnd/persistent/users/sungbino/tutorials/pycafana/prodgenie_cosmic_rockbox_sbnd_GenieGen-20250221T050016_G4-20250221T050844_1ffa1ec6-96f0-473c-918a-be174a12deb2__DetSim2-20250305T012350_Reco1-20250305T074914_Reco2-20250306T170024.flat.caf.root"
f = uproot.open(test_sample_path, timeout=120)

In [ ]:
mcdf = make_mcdf(f)
mcdf["ind"] = mcdf.index.get_level_values(1)
nuind = mcdf.ind

In [ ]:
nuind.value_counts()

In [ ]:
syst_branches = ["wgts.name", "wgts.type", "wgts.nuniv"]

In [ ]:
systdf = loadbranches(f["globalTree/global/wgts"], syst_branches)

In [ ]:
systdf

In [ ]:
systdf = systdf.reset_index().drop(columns =['entry', 'subentry'])
systdf = systdf.wgts

In [ ]:
print(systdf.head(60))

In [ ]:
systdf[(systdf['type']==3) & (systdf['nuniv']==1)]

In [ ]:
nuniv = systdf.nuniv.sum()
print(nuniv)

In [ ]:
wgtdf = loadbranches(f["recTree"], ["rec.mc.nu.wgt.univ"]).rec.mc.nu.wgt
wgtdf = wgtdf.rename(columns={"univ": "wgt"})

In [ ]:
wgtdf.head(50)

In [ ]:
wgtdf = wgtdf.rename_axis(['entry', 'rec.mc.nu..index', 'isyst', 'iuniv'])

In [ ]:
wgtdf = wgtdf.reset_index().set_index(['entry', 'rec.mc.nu..index'])

In [ ]:
wgtdf

In [ ]:
systs = []
for s in regen_systematics_sbnd:
    try:
        isyst = systdf.index[systdf['name'] == s][0]
    except IndexError:
        continue
    this_systs = []

    if systdf.type[isyst] == 3 and systdf.nuniv[isyst] == 1: # morph unisim                                                                                                                                
        s_morph = wgtdf[wgtdf.isyst == isyst].wgt.groupby(level=[0,1]).agg(lambda x: list(x))
        s_morph.name = (s)
        this_systs.append(s_morph)

    elif systdf.type[isyst] == 3 and systdf.nuniv[isyst] > 1: # +/- sigma unisim                                                                                                                           
        def insert_middle(lst, value):
            mid = len(lst) // 2
            return lst[:mid] + [value] + lst[mid:]

        s_nsigma = wgtdf[wgtdf.isyst == isyst].wgt.groupby(level=[0,1]).agg(
            lambda x: insert_middle(list(x), 1.)
        )   
        #s_nsigma = wgtdf[wgtdf.isyst == isyst].wgt.groupby(level=[0,1]).agg(lambda x: list(x))
        s_nsigma.name = (s)
        this_systs.append(s_nsigma)

    elif systdf.type[isyst] == 0: # multisim                                                                                                                                                               
        s_multisim = wgtdf[wgtdf.isyst == isyst].wgt.groupby(level=[0,1]).agg(lambda x: list(x)).head(250) # limit to 250 universes                                                                        
        s_multisim.name = (s)
        this_systs.append(s_multisim)

    else:
        raise Exception("Cannot decode systematic uncertainty: %s" % s)

    for syst in this_systs:
        systs.append(syst)

systs = pd.DataFrame(systs).T

In [ ]:
for idx, row in systs.GENIEReWeight_SBN_v1_multisigma_RPA_CCQE.loc[systs.index.get_level_values('entry') == 92].items():
    print(f"{idx}: {row}")

In [ ]:
systs

In [ ]:
systs.columns

In [ ]:
systs.columns = pd.MultiIndex.from_tuples([(col, '', '') for col in systs.columns])

In [ ]:
systs

In [ ]:
mcdf

In [ ]:
mcnuconcatdf = pd.concat([mcdf, systs], axis=1, verify_integrity=True)

In [ ]:
mcnuconcatdf